#  Choosing an ODE Algorithm

While the default algorithms, along with `alg_hints = [:stiff]`, will suffice in most cases, there are times when you may need to exert more control. The purpose of this part of the tutorial is to introduce you to some of the most widely used algorithm choices and when they should be used. The corresponding page of the documentation is the [ODE Solvers](http://docs.juliadiffeq.org/latest/solvers/ode_solve.html) page which goes into more depth.

## Diagnosing Stiffness

One of the key things to know for algorithm choices is whether your problem is stiff. Let's take for example the driven Van Der Pol equation:

In [1]:
using DifferentialEquations
van! = @ode_def VanDerPol begin
  dy = μ*((1-x^2)*y - x)
  dx = 1*y
end μ

prob = ODEProblem(van!,[0.0,2.0],(0.0,6.3),1e6)

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 6.3)
u0: [0.0, 2.0]

One indicating factor that should alert you to the fact that this model may be stiff is the fact that the parameter is `1e6`: large parameters generally mean stiff models. If we try to solve this with the default method:

In [2]:
sol = solve(prob)

retcode: MaxIters
Interpolation: specialized 4th order "free" interpolation
t: 999978-element Array{Float64,1}:
 0.0       
 4.9975e-10
 5.49725e-9
 3.28991e-8
 9.05558e-8
 1.73095e-7
 2.79375e-7
 4.14953e-7
 5.80791e-7
 7.8128e-7 
 1.01968e-6
 1.30218e-6
 1.6366e-6 
 ⋮         
 1.84586   
 1.84586   
 1.84586   
 1.84586   
 1.84586   
 1.84586   
 1.84587   
 1.84587   
 1.84587   
 1.84587   
 1.84587   
 1.84587   
u: 999978-element Array{Array{Float64,1},1}:
 [0.0, 2.0]          
 [-0.000998751, 2.0] 
 [-0.0109043, 2.0]   
 [-0.0626554, 2.0]   
 [-0.158595, 2.0]    
 [-0.270036, 2.0]    
 [-0.37832, 2.0]     
 [-0.474679, 2.0]    
 [-0.54993, 2.0]     
 [-0.602693, 2.0]    
 [-0.635376, 2.0]    
 [-0.653257, 2.0]    
 [-0.661747, 2.0]    
 ⋮                   
 [-0.777544, 1.83159]
 [-0.777545, 1.83159]
 [-0.777546, 1.83159]
 [-0.777547, 1.83159]
 [-0.777548, 1.83159]
 [-0.777549, 1.83159]
 [-0.77755, 1.83159] 
 [-0.777551, 1.83159]
 [-0.777552, 1.83158]
 [-0.777553, 1.83158]
 [-

Here it shows that maximum iterations were reached. Another thing that can happen is that the solution can return that the solver was unstable (exploded to infinity) or that `dt` became too small. If these happen, the first thing to do is to check that your model is correct. It could very well be that you made an error that causes the model to be unstable!

If the model is the problem, then stiffness could be the reason. We can thus hint to the solver to use an appropriate method:

In [3]:
sol = solve(prob,alg_hints = [:stiff])

retcode: Success
Interpolation: specialized 3rd order "free" stiffness-aware interpolation
t: 730-element Array{Float64,1}:
 0.0       
 4.9975e-10
 5.45414e-9
 1.89543e-8
 4.14966e-8
 7.30807e-8
 1.17146e-7
 1.74812e-7
 2.48623e-7
 3.40254e-7
 4.52108e-7
 5.86677e-7
 7.47363e-7
 ⋮         
 5.65145   
 5.66158   
 5.69681   
 5.74799   
 5.81057   
 5.88543   
 5.96795   
 6.05487   
 6.14133   
 6.22312   
 6.29638   
 6.3       
u: 730-element Array{Array{Float64,1},1}:
 [0.0, 2.0]          
 [-0.000998751, 2.0] 
 [-0.0108195, 2.0]   
 [-0.0368509, 2.0]   
 [-0.0780351, 2.0]   
 [-0.131248, 2.0]    
 [-0.19755, 2.0]     
 [-0.272074, 2.0]    
 [-0.350452, 2.0]    
 [-0.426453, 2.0]    
 [-0.494929, 2.0]    
 [-0.551972, 2.0]    
 [-0.59584, 2.0]     
 ⋮                   
 [0.667157, -1.99912]
 [0.670953, -1.99234]
 [0.684721, -1.96846]
 [0.706461, -1.93287]
 [0.736368, -1.88775]
 [0.778221, -1.8311] 
 [0.834755, -1.76464]
 [0.911736, -1.68889]
 [1.01723, -1.60577] 
 [1.16577, -1.51

Another way to understand stiffness is to look at the solution.

In [4]:
using Plots; gr()
sol = solve(prob,alg_hints = [:stiff],reltol=1e-6)
plot(sol,denseplot=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 -1000000 
 
 
 -500000 
 
 
 0 
 
 
 500000 
 
 
 1000000 
 
 
 t 
 
<polyline clip-path="url(#clip0702)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 68.7969,185.94 68.7969,185.94 68.7969,185.94 68.7969,185.94 68.7969,185.94 68.7969,185.94 68.7969,185.94 68.7969,185.94 68.7969,185.94 68.7969,185.94 
 68.7969,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 
 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.797,185.94 68.7971,185.94 
 68.7971,185.94 68.7971,185.94 68.7971,185.94 68.7971,185.94 68.7971,185.94 68.7971,185.94 68.7971,185.94 68.7972,185.94 68.7972,185.94 68.7972,185.94 
 68.7972,185.94 68.7973,185.94 68.7973,185.94 68.7974,185.94 68.7976,185.94 68.7978,185.94 68.7986,185.94 68.8012,185.94 68.826,185.94 68.9178,185.94 
 69.1859,185.94 69.7901,185.94 70.687,185.94 71.8239,185.94 73.2362,185.94 74.8803,185.94 76.7392,185.94 78.7671,185.94 80.9275,185.94 83.1778,185.94 
 85.4818,185.94 87.8053,185.94 90.1201,185.94 92.4027,185.94 94.6342,185.94 96.8003,185.94 98.8904,185.94 100.897,185.94 102.816,185.94 104.645,185.94 
 106.382,185.94 108.03,185.94 109.588,185.94 111.06,185.94 112.448,185.94 113.757,185.94 114.989,185.94 116.148,185.94 117.238,185.94 118.262,185.94 
 119.224,185.94 120.128,185.94 120.977,185.94 121.774,185.94 122.523,185.94 123.225,185.94 123.885,185.94 124.504,185.94 125.086,185.94 125.632,185.94 
 126.144,185.94 126.626,185.94 127.078,185.94 127.502,185.94 127.901,185.94 128.276,185.94 128.627,185.94 128.958,185.94 129.269,185.94 129.561,185.94 
 129.835,185.94 130.093,185.94 130.335,185.94 130.563,185.94 130.777,185.94 130.978,185.94 131.167,185.94 131.345,185.94 131.513,185.94 131.67,185.94 
 131.818,185.94 131.957,185.94 132.088,185.94 132.211,185.94 132.326,185.94 132.435,185.94 132.537,185.94 132.633,185.94 132.723,185.94 132.808,185.94 
 132.888,185.94 132.963,185.94 133.033,185.94 133.099,185.94 133.161,185.94 133.22,185.94 133.274,185.94 133.326,185.94 133.374,185.94 133.419,185.94 
 133.461,185.94 133.501,185.94 133.538,185.94 133.573,185.94 133.606,185.94 133.636,185.94 133.664,185.94 133.691,185.94 133.716,185.94 133.739,185.94 
 133.761,185.94 133.781,185.94 133.8,185.94 133.818,185.94 133.835,185.941 133.85,185.941 133.865,185.941 133.878,185.941 133.891,185.941 133.903,185.941 
 133.915,185.941 133.925,185.941 133.935,185.941 133.945,185.941 133.954,185.941 133.962,185.941 133.97,185.941 133.978,185.941 133.985,185.941 133.992,185.941 
 133.999,185.941 134.005,185.941 134.011,185.941 134.017,185.941 134.023,185.941 134.028,185.941 134.034,185.941 134.039,185.941 134.044,185.941 134.048,185.941 
 134.053,185.941 134.058,185.941 134.062,185.941 134.066,185.941 134.071,185.941 134.075,185.941 134.079,185.941 134.083,185.941 134.087,185.941 134.09,185.941 
 134.094,185.941 134.098,185.941 134.101,185.941 134.105,185.941 134.108,185.941 134.111,185.941 134.115,185.941 134.118,185.941 134.121,185.941 134.125,185.941 
 134.128,185.941 134.131,185.941 134.134,185.941 134.137,185.941 134.14,185.941 134.143,185.941 134.146,185.941 134.148,185.941 134.151,185.941 134.154,185.941 
 134.157,185.941 134.16,185.941 134.162,185.941 134.165,185.941 134.167,185.941 134.17,185.941 134.173,185.941 134.175,185.941 134.178,185.941 134.18,185.941 
 134.183,185.941 134.185,185.941 134.187,185.941 134.19,185.941 134.192,185.941 134.195,185.941 134.197,185.941 134.199,185.941 134.201,185.941 134.204,185.941 
 134.206,185.941 134.208,185.941 134.21,185.942 134.212,185.942 134.215,185.942 134.217,185.942 134.219,185.942 134.221,185.942 134.223,185.942 134.225,185.942 
 134.227,185.942 134.229,185.942 134.231,185.942 134.233,185.942 134.235,185.942 134

Let's zoom in on the y-axis to see what's going on:

In [5]:
plot(sol,ylims = (-10.0,10.0))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 t 
 
<polyline clip-path="url(#clip0902)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 35.8488,185.938 35.861,197.548 35.8732,197.549 35.8855,197.549 35.8977,197.55 35.91,197.551 35.9222,197.552 35.9344,197.553 35.9467,197.554 35.9589,197.555 
 35.9712,197.556 35.9834,197.557 35.9956,197.558 36.0079,197.559 36.0201,197.56 36.0323,197.56 36.0446,197.561 36.0568,197.562 36.0691,197.563 36.0813,197.564 
 36.0935,197.565 36.1058,197.566 36.118,197.567 36.1303,197.568 36.1425,197.569 36.1547,197.57 36.167,197.571 36.1792,197.571 36.1914,197.572 36.2037,197.573 
 36.2159,197.574 36.2282,197.575 36.2404,197.576 36.2526,197.577 36.2649,197.578 36.2771,197.579 36.2893,197.58 36.3016,197.581 36.3138,197.582 36.3261,197.582 
 36.3383,197.583 36.3505,197.584 36.3628,197.585 36.375,197.586 36.3873,197.587 36.3995,197.588 36.4117,197.589 36.424,197.59 36.4362,197.591 36.4484,197.592 
 36.4607,197.593 36.4729,197.594 36.4852,197.595 36.4974,197.595 36.5096,197.596 36.5219,197.597 36.5341,197.598 36.5464,197.599 36.5586,197.6 36.5708,197.601 
 36.5831,197.602 36.5953,197.603 36.6075,197.604 36.6198,197.605 36.632,197.606 36.6443,197.607 36.6565,197.607 36.6687,197.608 36.681,197.609 36.6932,197.61 
 36.7054,197.611 36.7177,197.612 36.7299,197.613 36.7422,197.614 36.7544,197.615 36.7666,197.616 36.7789,197.617 36.7911,197.618 36.8034,197.619 36.8156,197.62 
 36.8278,197.621 36.8401,197.621 36.8523,197.622 36.8645,197.623 36.8768,197.624 36.889,197.625 36.9013,197.626 36.9135,197.627 36.9257,197.628 36.938,197.629 
 36.9502,197.63 36.9625,197.631 36.9747,197.632 36.9869,197.633 36.9992,197.634 37.0114,197.635 37.0236,197.635 37.0359,197.636 37.0481,197.637 37.0604,197.638 
 37.0726,197.639 37.0848,197.64 37.0971,197.641 37.1093,197.642 37.1215,197.643 37.1338,197.644 37.146,197.645 37.1583,197.646 37.1705,197.647 37.1827,197.648 
 37.195,197.649 37.2072,197.65 37.2195,197.65 37.2317,197.651 37.2439,197.652 37.2562,197.653 37.2684,197.654 37.2806,197.655 37.2929,197.656 37.3051,197.657 
 37.3174,197.658 37.3296,197.659 37.3418,197.66 37.3541,197.661 37.3663,197.662 37.3785,197.663 37.3908,197.664 37.403,197.665 37.4153,197.666 37.4275,197.666 
 37.4397,197.667 37.452,197.668 37.4642,197.669 37.4765,197.67 37.4887,197.671 37.5009,197.672 37.5132,197.673 37.5254,197.674 37.5376,197.675 37.5499,197.676 
 37.5621,197.677 37.5744,197.678 37.5866,197.679 37.5988,197.68 37.6111,197.681 37.6233,197.682 37.6356,197.683 37.6478,197.684 37.66,197.684 37.6723,197.685 
 37.6845,197.686 37.6967,197.687 37.709,197.688 37.7212,197.689 37.7335,197.69 37.7457,197.691 37.7579,197.692 37.7702,197.693 37.7824,197.694 37.7946,197.695 
 37.8069,197.696 37.8191,197.697 37.8314,197.698 37.8436,197.699 37.8558,197.7 37.8681,197.701 37.8803,197.702 37.8926,197.703 37.9048,197.703 37.917,197.704 
 37.9293,197.705 37.9415,197.706 37.9537,197.707 37.966,197.708 37.9782,197.709 37.9905,197.71 38.0027,197.711 38.0149,197.712 38.0272,197.713 38.0394,197.714 
 38.0517,197.715 38.0639,197.716 38.0761,197.717 38.0884,197.718 38.1006,197.719 38.1128,197.72 38.1251,197.721 38.1373,197.722 38.1496,197.723 38.1618,197.724 
 38.174,197.725 38.1863,197.725 38.1985,197.726 38.2107,197.727 38.223,197.728 38.2352,197.729 38.2475,197.73 38.2597,197.731 38.2719,197.732 38.2842,197.733 
 38.2964,197.734 38.3087,197.735 38.3209,197.736 38.3331,197.737 38.3454,197.738 38.3576,197.739 38.3698,197.74 38.3821,197.741 38.3943,197.742 38.4066,197.743 
 38.4188,197.744 38.431,197.745 38.4433,197.746 38.4555,197.747 38.4678,197.748 38.48,197.749 38.4922,197.75 38.5045,197.751 38.5167,197.751 38.5289,197.752 
 38.5412,197.753 38.5534,197.754 38.5657,197.755 38.5779,197.756 38.5901,197.757 38.6024,197.758 38.6146,197.759 38.6268,197.76 38.6391,1

Notice how there are some extreme vertical shifts that occur. These vertical shifts are places where the derivative term is very large, and this is indicative of stiffness. This is an extreme example to highlight the behavior, but this general idea can be carried over to your problem. When in doubt, simply try timing using both a stiff solver and a non-stiff solver and see which is more efficient.

To try this out, let's use BenchmarkTools, a package that let's us relatively reliably time code blocks.

In [6]:
using BenchmarkTools

Let's compare the performance of non-stiff and stiff solvers on the Lorenz equation, which we saw in the last notebook, "ODEIntroduction".

First, let's grab all our information about the Lorenz equation from the last notebook.

In [7]:
function lorenz!(du,u,p,t)
    σ,ρ,β = p
    du[1] = σ*(u[2]-u[1])
    du[2] = u[1]*(ρ-u[3]) - u[2]
    du[3] = u[1]*u[2] - β*u[3]    
end
u0 = [1.0,0.0,0.0]
p = (10,28,8/3)
tspan = (0.0,100.0)
prob = ODEProblem(lorenz!,u0,tspan,p)

DiffEqBase.ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 100.0)
u0: [1.0, 0.0, 0.0]

And now, let's use the `@btime` macro from benchmark tools to compare the use of non-stiff and stiff solvers on this problem.

In [8]:
@btime solve(prob);

  1.136 ms (13385 allocations: 1.40 MiB)


In [9]:
@btime solve(prob,alg_hints = [:stiff]);

  8.222 ms (57126 allocations: 2.57 MiB)


In this particular case, we can see that non-stiff solvers get us to the solution much more quickly.

## The Recommended Methods

When picking a method, the general rules are as follows:

- Higher order is more efficient at lower tolerances, lower order is more efficient at higher tolerances
- Adaptivity is essential in most real-world scenarios
- Runge-Kutta methods do well with non-stiff equations, Rosenbrock methods do well with small stiff equations, BDF methods do well with large stiff equations

While there are always exceptions to the rule, those are good guiding principles. Based on those, a simple way to choose methods is:

- The default is `Tsit5()`, a non-stiff Runge-Kutta method of Order 5
- If you use low tolerances (`1e-8`), try `Vern7()` or `Vern9()`
- If you use high tolerances, try `BS3()`
- If the problem is stiff, try `Rosenbrock23()`, `Rodas4()`, or `CVODE_BDF()`

(This is a simplified version of the default algorithm chooser)

## Comparison to other Software

If you are familiar with MATLAB, SciPy, or R's DESolve, here's a quick translation start to have transfer your knowledge over.

- ode23 –> BS3()
- ode45/dopri5 –> DP5(), though in most cases Tsit5() is more efficient
- ode23s –> Rosenbrock23(), though in most cases Rodas4() is more efficient
- ode113 –> CVODE_Adams(), though in many cases Vern7() is more efficient
- dop853 –> DP8(), though in most cases Vern7() is more efficient
- ode15s/vode –> CVODE_BDF(), though in many cases Rodas4() or radau() are more efficient
- ode23t –> Trapezoid() for efficiency and GenericTrapezoid() for robustness
- ode23tb –> TRBDF2
- lsoda –> lsoda() (requires Pkg.add("LSODA"); using LSODA)
- ode15i –> IDA(), though in many cases Rodas4() can handle the DAE and is significantly more efficient